# BAT.jl Tutorial - Poisson Counting Experiment

In [ ]:
using BAT
using Distributions 
using IntervalSets
using ValueShapes
using Plots
using ArraysOfArrays
using StatsBase 
pyplot();

## The Situation
We want to determine the properties of a radioactive source.
Due to natural sources of radioactivity we need to take into account the contamination of this background to our signal. 

## 1. Background only measurement
We start by using our detector without the source installed.
This measurement yields a number of $N_B=10$ counts.
Using this measurement we want to gain information about the event rate of the natural radioactive background.

### Task: 
Perform a Bayesian analysis to estimate the event rate of the natural background $\lambda_b$ using a Poisson model.
Start by defining the model and its Likelihood using the *logpdf()* and *Poisson()* functions of the distrtibution package.

In [ ]:
struct Background<:AbstractDensity
    k::Float64 # observed counts
end

function BAT.density_logval(target::Background, params::Union{NamedTuple,AbstractVector{<:Real}})
    return logpdf(Poisson(params.λb), target.k) # poisson log-likelihood
end

Create an instance of the model and define the Prior with help of the *Named Prior()* function.

Afterwards, use the model and the prior to define the *PosteriorDensity()*. 

In [ ]:
# Number of observed background events
kb = 10
likelihood_B = Background(kb)

prior_B = NamedPrior(
    λb = 0..30
)

posterior_B = PosteriorDensity(likelihood_B, prior_B);

Define the settings for the sampling. Choose *MetropolisHstings()* as your algorithm and set the number of chains and samples.

In [ ]:
algorithm = MetropolisHastings()
nchains = 8
nsamples = 10^5;

Start the sampling by using *rand()* on the *MCMCSpec()* object using the settings defined above.

In [ ]:
samples_B, sampleids_B, stats_B, chains_B = rand(MCMCSpec(algorithm, posterior_B), nsamples, nchains);

At last look at the resulting disribution for the background rate using *plot()* and print out the reults of the sampling. 

In [ ]:
plot(posterior_B, samples_B, :λb)
plot!(prior_B, :λb, xlabel = "\$\\lambda_b\$", ylabel = "\$P(\\lambda_b)\$")

In [ ]:
println("Mode: $(stats_B.mode)")
println("Mean: $(stats_B.param_stats.mean)")
#println("Covariance: $(stats_B.param_stats.cov)")
println("Standard Deviation: $([sqrt((stats_B.param_stats.cov)[i,i]) for i in 1:(Int(sqrt(length(stats_B.param_stats.cov))))])")

## 2. Further Background only measurement
A second measurement of the natural background yields a number of $N_B=8$ counts.
Therefore, we want to update our estimation for the background rate using this new knowledge and the first result.
### Task:
Perform an anylsis in a similar fashion to the first one with the posterior distribution of the background measurement as the prior of this analysis.
This can be done by using a *StatsBase* histogram using *fit(Histogram,flatview(samples)[i,:],weights,nbins)*.

Be mindful about carrying on the weights of the samples using the *FrequencyWeights()* function on the samples. 

In [ ]:
posterior_hist_B1 = fit(Histogram, flatview(samples_B.params)[1, :], FrequencyWeights(samples_B.weight), nbins = 400, closed = :left);

The histogram can be used as prior by converting it into a univariate distribution using *BAT.HistogramAsUvDistribution()*.
Otherwise proceed similarly to the first task.

In [ ]:
# Number of observed background events
kb2 = 8
likelihood_B2 = Background(kb2)

prior_B2 = NamedPrior(
    λb = BAT.HistogramAsUvDistribution(posterior_hist_B1) # replace by analytic poisson prior
)

prior_B2flat = NamedPrior(
    λb = 0..30
)

posterior_B2 = PosteriorDensity(likelihood_B2, prior_B2)
likelipost_B2 = PosteriorDensity(likelihood_B2, prior_B2flat)
;

In [ ]:
samples_B2, sampleids_B2, stats_B2, chains_B2 = rand(MCMCSpec(algorithm, posterior_B2), nsamples, nchains);
likelipost_samples_B2, likelids_B2, likeli_stats_B2, like_chains_B2 = rand(MCMCSpec(algorithm, likelipost_B2), nsamples, nchains);

In [ ]:
println("Mode: $(stats_B2.mode)")
println("Mean: $(stats_B2.param_stats.mean)")
#println("Covariance: $(stats_B2.param_stats.cov)")
println("Standard Deviation: $([sqrt((stats_B2.param_stats.cov)[i,i]) for i in 1:(Int(sqrt(length(stats_B2.param_stats.cov))))])")

Use the *plot!(prior)* function to visulaize both the posterior of the first analysis and the updated posterior. 

In [ ]:
plot(posterior_B2, samples_B2, :λb)
#plot!(likelipost_B2, likelipost_samples_B2, :λb, seriestype=:stephist, linecolor=:blue,linewidth=1.5, localmode=false, label="likelihood")
plot!(prior_B2, :λb, linewidth=1.5, xlabel = "\$\\lambda_b\$", ylabel = "\$P(\\lambda_b)\$")

## 3. Signal + Background
Including the radioactive source to our experimental setup we repeat our measurement and obtain $N_{S+B}=12$ counts.
With this measurement and our prior knowledge we are able to estimate the rate of the signal $\lambda_s$.
### Task
Perform a third analysis using a poisson model with the combined singal + background rate.
Use the known information about the background as a prior and choose a suitable prior for the signal.

In [ ]:
struct SignalAndBackground<:AbstractDensity
    k::Float64 # observed counts
end

function BAT.density_logval(target::SignalAndBackground, params::Union{NamedTuple,AbstractVector{<:Real}})
    return logpdf(Poisson(params.λb + params.λs), target.k)  # poisson log-likelihood
end

kSB = 12
likelihood_SB = SignalAndBackground(kSB);

In [ ]:
hist_B2 = fit(Histogram, flatview(samples_B2.params)[1, :], FrequencyWeights(samples_B2.weight), nbins = 400, closed = :left)
B2 = BAT.HistogramAsUvDistribution(hist_B2);

In [ ]:
prior_SB = NamedPrior(
    λb = B2,
    λs = 0..30
)

posterior_SB = PosteriorDensity(likelihood_SB, prior_SB);

In [ ]:
samples_SB, sampleids_SB, stats_SB, chains_SB = rand(MCMCSpec(algorithm, posterior_SB), nsamples, nchains);

In [ ]:
plot(samples_SB,param_labels=["\$\\lambda_b\$","\$\\lambda_s\$"])

In [ ]:
println("Mode: $(stats_SB.mode)")
println("Mean: $(stats_SB.param_stats.mean)")
#println("Covariance: $(stats_SB.param_stats.cov)")
println("Standard Deviation: $([sqrt((stats_SB.param_stats.cov)[i,i]) for i in 1:(Int(sqrt(length(stats_SB.param_stats.cov))))])")

Questions:
* (How) Does the distribtion of the background rate change?
* How would you communicate your result of the signal rate (estimate value and uncertainty)? 

## 4. Error propagation

Finally, we want to caluclate the cross section of the signal process.
The rate of measured events in a detector of a couting experiment can be written as 

$\frac{\mathrm d N}{\mathrm d t} = ε \cdot σ \cdot L$

with the Luminosity $L$ and the efficiency of our detector $ε$. Using this the signal cross section
### $σ_S = \frac{λ_s}{ε \cdot L}$
can be derived. For this experiment the value of $L$ set to $1.1$.

As a final result we should obtain either a measurement or an upper limit on the signal cross section.

### Task a) Known efficiency with gaussian uncertainty
The efficiency has been measured to be $ε = 0.1 \pm 0.02$.
Assume the error to follow a normal distribution.

Calculate the signal cross section $σ_S$ using the equation above.

Use the *Distributions* package and *rand()* to obain a sample for $\epsilon$.
In order to obtain unweigthed samples of $\lambda_S$ the function *sample(array,weights,n,ordered=false,replace=false)* can be used.

The function *broadcast()* might be useful for element wise operation when handeling the samples.

In [ ]:
ε    = rand(Normal(0.1,0.02),nsamples)
λ_SB = sample(flatview(samples_SB.params)[2,:],FrequencyWeights(samples_SB.weight),nsamples,ordered=false,replace=false)
L = 1.1
σS = (λ_SB)./(ε*L);

In [ ]:
hist_σ = fit(Histogram, σS,nbins=200,closed = :left)
plot(hist_σ,1,seriestype=:smallest_intervals,xlim=(0,400),xlabel="\$\\sigma_s\$",ylabel="\$P(\\sigma_s)\$")

Questions:
* What is the 95% upper limit on the crosssection estimated from the distribution?

### Task b) Binomial analysis of calibration measurement with known source 
The number of expected events is $N_\text{expected} = 200$.
The detector measures $N_\text{measured} = 21$ events.
Implement a binomial model using the *Binomial(n,p)* function of the Distributions package and extract the efficiency of the detector with BAT.
Afterwards, repeat the calculations in a) using the posterior distrtibuion of the efficiency.

In [ ]:
struct BinomialModel<:AbstractDensity
    n::Int64 # n trials
    k::Int64 # k succes
end

function BAT.density_logval(target::BinomialModel, params::Union{NamedTuple,AbstractVector{<:Real}})
    return logpdf(Binomial(target.n, params.p), target.k) # poisson log-likelihood
end

likelihood_binomial = BinomialModel(200, 21)

In [ ]:
prior_binomial = NamedPrior(
    p = 0..1
)
posterior_binomial = PosteriorDensity(likelihood_binomial, prior_binomial);

In [ ]:
samples_binomial, sampleids_binomial, stats_binomial, chains_binomial = rand(MCMCSpec(algorithm, posterior_binomial), nsamples, nchains);

In [ ]:
plot(samples_binomial, 1,xlabel="\$\\epsilon\$",ylabel="\$P(\\epsilon)\$")

In [ ]:
println("Mode: $(stats_binomial.mode)")
println("Mean: $(stats_binomial.param_stats.mean)")
#println("Covariance: $(stats_binomial.param_stats.cov)")
println("Standard Deviation: $([sqrt((stats_binomial.param_stats.cov)[i,i]) for i in 1:(Int(sqrt(length(stats_binomial.param_stats.cov))))])")

In [ ]:
λS  = sample(samples_SB.params.data[2,:],FrequencyWeights(samples_SB.weight),60000,replace=false,ordered=false)
ε   = sample(samples_binomial.params.data[1,:],FrequencyWeights(samples_binomial.weight),60000,replace=false,ordered=false)
σS  = λS./(ε*L)
hist_σ = fit(Histogram, σS,nbins=200,closed = :left)
plot(hist_σ,1,seriestype=:smallest_intervals,xlabel="\$\\sigma_s\$",ylabel="\$P(\\sigma_s)\$")

In [ ]:
plot(hist_σ,1,seriestype=:smallest_intervals,xlabel="\$\\sigma_s\$",ylabel="\$P(\\sigma_s)\$",intervals=[0.90,0.95,0.99]) #custom Intervals

Questions:
* What is the 95%(90%) upper limit on the cross section?